In [21]:
import pandas as pd
import os
import numpy as np

In [22]:
class DataLoader():
    def __init__(self):
        self._data_path = "../../ieee-fraud-detection/"

    def loadCsv(self, file_name):
        with open(os.path.join(self._data_path, file_name)) as f:
            csv = pd.read_csv(f)
    
        return csv

In [36]:
dl = DataLoader()
df_id = dl.loadCsv("train_identity.csv")
df_tr = dl.loadCsv("train_transaction.csv")

## Transaction Table
IP, ISP, Proxy, UA, browser, os, etc.associated with the transactions. Field names are masked for privacy

**Cateogorical features**: DeviceType, DeviceInfo, id_12-id_38

In [24]:
df_id
# Some of these variables may or may not be useful. id_33 shows resolution. Low resolution could be correlated with fraud, as older less secure devices will generally have poorer resolution

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 66.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,F3111 Build/33.3.A.1.97
144229,3577526,-5.0,172059.0,NaN,NaN,1.0,-5.0,NaN,NaN,NaN,...,chrome 55.0 for android,32.0,855x480,match_status:2,T,F,T,F,mobile,A574BL Build/NMF26F
144230,3577529,-20.0,632381.0,NaN,NaN,-1.0,-36.0,NaN,NaN,NaN,...,chrome 65.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,Moto E (4) Plus Build/NMA26.42-152
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,NaN,NaN,0.0,...,chrome 66.0,24.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [25]:
# we see that id_31 is a string with formatting browser + version + OS (sometimes)
# Let's extract the version and browser to use as separate variables
pattern = r'^(.*?)(\d+(?:\.\d+)*)(.*)$'
df_id[['browser', 'version', 'platform']] = df_id['id_31'].str.extract(pattern, expand=True)

# Now let's get all of those that don't include a version number
pattern = r'^([^0-9]*)$'
df_tmp = pd.DataFrame()
df_tmp["browser"] = df_id['id_31'].str.extract(pattern, expand=True)
df_tmp.dropna(inplace=True)

# add these back into the original dataframe
df_id.loc[df_id["browser"].isnull(), "browser"] = df_tmp

# drop the old column
df_id.drop("id_31", inplace=True, axis=1)

In [26]:
# let's see which rows contain useful data
for col in df_id.columns:
    print( (len(df_id) - df_id[col].isna().sum()) / len(df_id) *100, f" % of rows contain values for {col}")

# We should throw out anything that is sparesely populated or replace NaN with dummy values

100.0  % of rows contain values for TransactionID
100.0  % of rows contain values for id_01
97.66974270797945  % of rows contain values for id_02
45.983928781901504  % of rows contain values for id_03
45.983928781901504  % of rows contain values for id_04
94.8915990099353  % of rows contain values for id_05
94.8915990099353  % of rows contain values for id_06
3.5740780542594273  % of rows contain values for id_07
3.5740780542594273  % of rows contain values for id_08
51.94788987263663  % of rows contain values for id_09
51.94788987263663  % of rows contain values for id_10
97.74323490463348  % of rows contain values for id_11
100.0  % of rows contain values for id_12
88.27383469802334  % of rows contain values for id_13
55.49631499032815  % of rows contain values for id_14
97.74808816290307  % of rows contain values for id_15
89.67434637010948  % of rows contain values for id_16
96.62767882523417  % of rows contain values for id_17
31.27786290238711  % of rows contain values for id_18


TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
TransactionAMT: transaction payment amount in USD

ProductCD: product code, the product for each transaction

card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.

addr: address

dist: distance

P_ and (R__) emaildomain: purchaser and recipient email domain

C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.

D1-D15: timedelta, such as days between previous transaction, etc.

M1-M9: match, such as names on card and address, etc.

Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

Categorical Features:
ProductCD, 
card1 - card6, 
addr1, addr2, 
P_emaildomain, 
R_emaildomain, 
M1 - M9

In [27]:
df_tr.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


In [28]:
# Find correlations with fraud
corr_fraud = df_tr.corrwith(df_tr["isFraud"], numeric_only=True)

# Find least correlated variables
corr_fraud = np.abs(corr_fraud).sort_values(ascending=True)

# are these low correlations because of NaN or because no correlation exists?
# Let's find a way to 

In [29]:
# Organize columns which are equally sparsley populated
cols = []
col_dict = {}
for col in df_tr.columns:
    percent_pop = (len(df_tr) - df_tr[col].isna().sum()) / len(df_tr) *100 # percent of rows which are not NaN
    percent_pop = round(percent_pop, 2)
    
    if not percent_pop in col_dict.keys():
        print(percent_pop, "not in keys")
        col_dict[percent_pop] = []

    col_dict[percent_pop].append(col)
    print( percent_pop, f" % of rows contain values for {col}")

100.0 not in keys
100.0  % of rows contain values for TransactionID
100.0  % of rows contain values for isFraud
100.0  % of rows contain values for TransactionDT
100.0  % of rows contain values for TransactionAmt
100.0  % of rows contain values for ProductCD
100.0  % of rows contain values for card1
98.49 not in keys
98.49  % of rows contain values for card2
99.73 not in keys
99.73  % of rows contain values for card3
99.73  % of rows contain values for card4
99.28 not in keys
99.28  % of rows contain values for card5
99.73  % of rows contain values for card6
88.87 not in keys
88.87  % of rows contain values for addr1
88.87  % of rows contain values for addr2
40.35 not in keys
40.35  % of rows contain values for dist1
6.37 not in keys
6.37  % of rows contain values for dist2
84.01 not in keys
84.01  % of rows contain values for P_emaildomain
23.25 not in keys
23.25  % of rows contain values for R_emaildomain
100.0  % of rows contain values for C1
100.0  % of rows contain values for C2
1

In [53]:
# Look at correlation between variables which are sparsely populated
"""
Many variables are sparsley populated. And many appear to be sparsley populated in the same way.
For variables which are sparsley populated in the same way, find out which ones are correlated with each other.
Of the ones which are highly correlated, only keep the one which is most highly correlated with isFraud.
Alternatively: Only keep the one which  has the most unique values
"""
vars_to_keep = []
for percent in col_dict.keys():
    if percent < 50 and len(col_dict[percent]) > 1:
        corr_sparse = df_tr[col_dict[percent]].corr(numeric_only=True)    

        strong_corr = corr_sparse.where((corr_sparse.abs() > 0.75) & (corr_sparse.abs() < 1.0))
        # Do we want to keep sparsley popoulated variables if they aren't correlated with other variables?
        # nan_columns = strong_corr.columns[strong_corr.isna().all()]
        # vars_to_keep += list(nan_columns)
        
        strong_corr_pairs = strong_corr.stack().reset_index()
        strong_corr_pairs.columns = ['Variable 1', 'Variable 2', 'Correlation']
        if len(strong_corr_pairs) == 0: continue
        # remove duplicate pairs
        strong_corr_pairs = strong_corr_pairs[strong_corr_pairs['Variable 1'] < strong_corr_pairs['Variable 2']]
        
        # Find variable most highly correlated with isFraud
        vars_with_strong_corr = pd.unique(
            strong_corr_pairs[['Variable 1', 'Variable 2']].values.ravel()
        )
        corr_fraud = df_tr[vars_with_strong_corr].corrwith(df_tr["isFraud"], numeric_only=True).abs()

        vars_to_keep.append(corr_fraud.idxmax())
    else:
        vars_to_keep += col_dict[percent]

In [45]:
f = df_tr.corrwith(df_tr["D9"], numeric_only=True)
np.abs(f).sort_values(ascending=False)

/home/alex/miniconda3/envs/fraudsters/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/alex/miniconda3/envs/fraudsters/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


D9       1.000000
card3    0.102086
V94      0.101789
V33      0.081018
V15      0.080027
           ...   
V76           NaN
V90           NaN
V91           NaN
V107          NaN
V305          NaN
Length: 380, dtype: float64

In [31]:
df_tr_keep = df_tr[vars_to_keep]

In [32]:
corr_sparse

,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
V322,1.000000,0.944680,0.936062,0.111617,0.661089,0.682219,0.732846,0.848524,0.781830,0.986563,0.942703,0.932696,0.014451,0.568194,0.351582,0.181823,0.678443,0.471132
V323,0.944680,1.000000,0.992720,0.188827,0.745004,0.762645,0.764127,0.950547,0.874700,0.933984,0.994990,0.986371,0.024835,0.633840,0.389819,0.188245,0.759181,0.525305
V324,0.936062,0.992720,1.000000,0.212119,0.748075,0.774468,0.795705,0.960935,0.905371,0.924049,0.986414,0.991333,0.028794,0.639864,0.397213,0.196525,0.767756,0.544056
V325,0.111617,0.188827,0.212119,1.000000,0.459117,0.554233,0.329034,0.303774,0.347768,0.117165,0.181549,0.207172,0.196324,0.359131,0.303116,0.085592,0.240886,0.209176
V326,0.661089,0.745004,0.748075,0.459117,1.000000,0.937717,0.692267,0.796489,0.777691,0.651796,0.724974,0.733252,0.049735,0.640061,0.381933,0.164347,0.617962,0.455429
V327,0.682219,0.762645,0.774468,0.554233,0.937717,1.000000,0.738450,0.826449,0.821201,0.674547,0.746456,0.760725,0.076589,0.661806,0.435071,0.178088,0.648252,0.485927
V328,0.732846,0.764127,0.795705,0.329034,0.692267,0.738450,1.000000,0.867310,0.931738,0.727812,0.756865,0.786823,0.045242,0.575295,0.370373,0.299599,0.721737,0.601530
V329,0.848524,0.950547,0.960935,0.303774,0.796489,0.826449,0.867310,1.000000,0.969601,0.840474,0.941426,0.950356,0.040477,0.667052,0.416870,0.221552,0.799920,0.586808
V330,0.781830,0.874700,0.905371,0.347768,0.777691,0.821201,0.931738,0.969601,1.000000,0.772997,0.863443,0.891457,0.046645,0.647561,0.412448,0.245626,0.780555,0.611463
V331,0.986563,0.933984,0.924049,0.117165,0.651796,0.674547,0.727812,0.840474,0.772997,1.000000,0.947138,0.941697,0.060796,0.593782,0.389508,0.309989,0.752075,0.572187


In [33]:
strong_corr

,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
V322,NaN,0.944680,0.936062,NaN,NaN,NaN,NaN,0.848524,0.781830,0.986563,0.942703,0.932696,NaN,NaN,NaN,NaN,NaN,NaN
V323,0.944680,NaN,0.992720,NaN,NaN,0.762645,0.764127,0.950547,0.874700,0.933984,0.994990,0.986371,NaN,NaN,NaN,NaN,0.759181,NaN
V324,0.936062,0.992720,NaN,NaN,NaN,0.774468,0.795705,0.960935,0.905371,0.924049,0.986414,0.991333,NaN,NaN,NaN,NaN,0.767756,NaN
V325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V326,NaN,NaN,NaN,NaN,NaN,0.937717,NaN,0.796489,0.777691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V327,NaN,0.762645,0.774468,NaN,0.937717,NaN,NaN,0.826449,0.821201,NaN,NaN,0.760725,NaN,NaN,NaN,NaN,NaN,NaN
V328,NaN,0.764127,0.795705,NaN,NaN,NaN,NaN,0.867310,0.931738,NaN,0.756865,0.786823,NaN,NaN,NaN,NaN,NaN,NaN
V329,0.848524,0.950547,0.960935,NaN,0.796489,0.826449,0.867310,NaN,0.969601,0.840474,0.941426,0.950356,NaN,NaN,NaN,NaN,0.799920,NaN
V330,0.781830,0.874700,0.905371,NaN,0.777691,0.821201,0.931738,0.969601,NaN,0.772997,0.863443,0.891457,NaN,NaN,NaN,NaN,0.780555,NaN
V331,0.986563,0.933984,0.924049,NaN,NaN,NaN,NaN,0.840474,0.772997,NaN,0.947138,0.941697,NaN,NaN,NaN,NaN,0.752075,NaN


In [34]:
vars_to_keep

['TransactionID',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'V279',
 'V280',
 'V284',
 'V285',
 'V286',
 'V287',
 'V290',
 'V291',
 'V292',
 'V293',
 'V294',
 'V295',
 'V297',
 'V298',
 'V299',
 'V302',
 'V303',
 'V304',
 'V305',
 'V306',
 'V307',
 'V308',
 'V309',
 'V310',
 'V311',
 'V312',
 'V316',
 'V317',
 'V318',
 'V319',
 'V320',
 'V321',
 'card2',
 'card3',
 'card4',
 'card6',
 'card5',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'D1',
 'V281',
 'V282',
 'V283',
 'V288',
 'V289',
 'V296',
 'V300',
 'V301',
 'V313',
 'V314',
 'V315',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D10',
 'D11',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',


In [35]:
# Categorical encoding
"""
which cateogories should be one-hot-encoded vs. another method such as frequency encoding?
"""

'\nwhich cateogories should be one-hot-encoded vs. another method such as frequency encoding?\n'